In [2]:
import tkinter as tk
from tkinter import ttk, messagebox
import csv

class QChat10GUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Q-Chat-10 Toddler Features")

        # Create frames for each section
        self.feature_frame = tk.Frame(self.root)
        self.feature_frame.pack(fill="x")

        self.demographic_frame = tk.Frame(self.root)
        self.demographic_frame.pack(fill="x")

        self.score_frame = tk.Frame(self.root)
        self.score_frame.pack(fill="x")

        # Create labels and entries for features
        self.feature_labels = []
        self.feature_entries = []
        questions = [
            "A1: Does your child look at you when you call his/her name?",
            "A2: How easy is it for you to get eye contact with your child?",
            "A3: Does your child point to indicate that s/he wants something?",
            "A4: Does your child point to share interest with you?",
            "A5: Does your child pretend?",
            "A6: Does your child follow where you’re looking?",
            "A7: If you or someone else in the family is visibly upset, does your child show signs of wanting to comfort them?",
            "A8: Would you describe your child’s first words as:",
            "A9: Does your child use simple gestures?",
            "A10: Does your child stare at nothing with no apparent purpose?"
        ]
        for i, question in enumerate(questions):
            label = tk.Label(self.feature_frame, text=question)
            label.grid(row=i, column=0)
            self.feature_labels.append(label)

            entry = ttk.Combobox(self.feature_frame, values=["Always", "Usually", "Sometimes", "Rarely", "Never"])
            entry.grid(row=i, column=1)
            self.feature_entries.append(entry)

        # Create labels and entries for demographics
        self.demographic_labels = []
        self.demographic_entries = []
        demographic_features = ["Age", "Sex", "Jaundice", "Family_mem_with_ASD"]
        for i, feature in enumerate(demographic_features):
            label = tk.Label(self.demographic_frame, text=feature)
            label.grid(row=i, column=0)
            self.demographic_labels.append(label)

            if feature == "Sex":
                entry = ttk.Combobox(self.demographic_frame, values=["Male", "Female"])
            elif feature == "Jaundice":
                entry = ttk.Combobox(self.demographic_frame, values=["Yes", "No"])
            elif feature == "Family_mem_with_ASD":
                entry = ttk.Combobox(self.demographic_frame, values=["Yes", "No"])
            else:
                entry = tk.Entry(self.demographic_frame)
            entry.grid(row=i, column=1)
            self.demographic_entries.append(entry)

        # Create dropdown for Ethnicity
        self.ethnicity_label = tk.Label(self.demographic_frame, text="Ethnicity")
        self.ethnicity_label.grid(row=len(demographic_features), column=0)
        self.ethnicity_entry = ttk.Combobox(self.demographic_frame, values=[
            "Latino", "Native Indian", "Pacifica", "White European", "asian", "black", "middle eastern", "mixed", "south asian","Others"
        ])
        self.ethnicity_entry.grid(row=len(demographic_features), column=1)

        # Create dropdown for Who completed the test
        self.test_completed_label = tk.Label(self.demographic_frame, text="Who completed the test")
        self.test_completed_label.grid(row=len(demographic_features) + 1, column=0)
        self.test_completed_entry = ttk.Combobox(self.demographic_frame, values=[
            "Health care professional", "Others", "Self", "family member"
        ])
        self.test_completed_entry.grid(row=len(demographic_features) + 1, column=1)

        # Create button to calculate score
        self.calculate_button = tk.Button(self.score_frame, text="Calculate Score", command=self.calculate_score)
        self.calculate_button.pack()

        # Create button to save to CSV
        self.save_button = tk.Button(self.score_frame, text="Save to CSV", command=self.save_to_csv)
        self.save_button.pack()

        # Create label to display score
        self.score_label = tk.Label(self.score_frame, text="Q-Chat-10 Score: ")
        self.score_label.pack()

    def calculate_score(self):
        score = 0
        for i, entry in enumerate(self.feature_entries):
            response = entry.get()
            if i < 9:
                if response in ["Sometimes", "Rarely", "Never"]:
                    score += 1
            else:
                if response in ["Always", "Usually", "Sometimes"]:
                    score += 1
        
        # Update the score label with the calculated score
        self.score_label.config(text=f"Q-Chat-10 Score: {score}")
        
        # Determine ASD traits classification
        if score > 3:
            self.asd_traits_class = "Yes"
        else:
            self.asd_traits_class = "No"

    def save_to_csv(self):
        # Prepare data for CSV
        data = {}
        for i, entry in enumerate(self.feature_entries):
            question = f"A{i+1}"
            response = entry.get()
            # Assign "1" or "0" based on the response
            if i < 9:
                data[question] = "1" if response in ["Sometimes", "Rarely", "Never"] else "0"
            else:
                data[question] = "1" if response in ["Always", "Usually", "Sometimes"] else "0"

        # Demographic data
        demographic_keys = ["Age", "Sex", "Jaundice", "Family_mem_with_ASD"]
        demographic_entries = self.demographic_entries
        
        for key, entry in zip(demographic_keys, demographic_entries):
            data[key] = entry.get()

        # One-hot encoding for Ethnicity
        ethnicities = [
            "Latino", "Native Indian", "Others", "Pacifica", "White European", 
            "asian", "black", "middle eastern", "mixed", "south asian"
        ]
        selected_ethnicity = self.ethnicity_entry.get()
        for ethnicity in ethnicities:
            data[f"Ethnicity_{ethnicity}"] = "1" if selected_ethnicity == ethnicity else "0"

        # One-hot encoding for "Who completed the test"
        test_completed_options = [
            "Health care professional", "Others", "Self", "family member"
        ]
        selected_completed_by = self.test_completed_entry.get()
        for option in test_completed_options:
            data[f"Who completed the test_{option}"] = "1" if selected_completed_by == option else "0"

        # Add score and classification to data
        score_text = self.score_label.cget("text").split(": ")[1]
        data["Qchat-10-Score"] = score_text
        
        # Modify Class/ASD Traits to be "1" for "Yes" and "0" for "No"
        data["Class/ASD Traits"] = "1" if self.asd_traits_class == "Yes" else "0"

        # Write to CSV
        with open('qchat10_results.csv', 'a', newline='') as csvfile:
            fieldnames = [
                'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'Age',
                'Qchat-10-Score', 'Sex', 'Jaundice', 'Family_mem_with_ASD', 'Class/ASD Traits',
                'Ethnicity_Latino', 'Ethnicity_Native Indian', 'Ethnicity_Others', 'Ethnicity_Pacifica',
                'Ethnicity_White European', 'Ethnicity_asian', 'Ethnicity_black', 'Ethnicity_middle eastern',
                'Ethnicity_mixed', 'Ethnicity_south asian', 'Who completed the test_Health care professional',
                'Who completed the test_Others', 'Who completed the test_Self', 'Who completed the test_family member'
            ]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            # Write header if the file is empty
            if csvfile.tell() == 0:
                writer.writeheader()

            writer.writerow(data)

        # Notify user
        messagebox.showinfo("Saved", "Data saved to qchat10_results.csv")


# Initialize the main window
root = tk.Tk()
gui = QChat10GUI(root)
root.mainloop()


In [8]:
import pandas as pd

In [9]:
df = pd.read_csv('qchat10_results.csv')

In [10]:
df

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,Age,Sex,Jaundice,Family_mem_with_ASD,Ethnicity,Who_completed_test,Score
0,0,0,0,0,0,0,0,0,0,1,18,Male,No,No,Native Indian,Self,NaN
